In [ ]:
%pip install transformers

ERROR: Operation cancelled by user


import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments, GPT2Config
from google.colab import files

# Upload a file to Colab
uploaded = files.upload()

# Download a file from Colab
files.download('booksummaries.txt')

# Load pre-trained GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})  # Add padding token

# Read the book summaries dataset
with open("booksummaries.txt", "r", encoding="utf-8") as file:
    book_summaries = file.readlines()

# Tokenize and encode the book summaries
tokenized_datasets = tokenizer(
    book_summaries,
    truncation=True,
    padding=True,  # Enable padding
    return_tensors="pt"
)

# Create a PyTorch dataset
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="booksummaries.txt",  # Path to the file containing the text
    block_size=128  # Define the block size for tokenization
)

# Initialize the model configuration
model_config = GPT2Config.from_pretrained("gpt2")

# Load pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2", config=model_config)

# Fine-tune the model on the book summaries dataset
training_args = TrainingArguments(
    output_dir="./fine-tuned-model",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=100,
    do_train=True
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    train_dataset=dataset
)

# Start training
trainer.train()

# Define function to generate story based on user prompt
def generate_story(user_prompt, max_length=50, temperature=0.7, top_k=50):
    # Tokenize and encode user prompt
    input_ids = tokenizer.encode(user_prompt, return_tensors="pt")

    # Generate story based on user prompt
    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        temperature=temperature,
        top_k=top_k,
        repetition_penalty=1.2  # Add a repetition penalty to encourage diversity
    )

    # Decode generated story
    generated_story = tokenizer.decode(output[0], skip_special_tokens=True)

    return generated_story

# Prompt user for input and generate story
user_prompt = input("Enter your story prompt: ")
generated_story = generate_story(user_prompt)

# Display generated story
print("Generated Story:")
print(generated_story)


In [1]:
%pip install transformers[torch] accelerate


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
data = pd.read_csv("Dialogue.csv",encoding="latin1")
print(data.head())
#expanding the dispay of text sms column
pd.set_option('display.max_colwidth', -1)
#using only v1 and v2 column
data= data [['Dialogue']]
data.head()

   Dialogue ID  Chapter ID  Place ID  Character ID  \
0            1           1         8             4   
1            2           1         8             7   
2            3           1         8             4   
3            4           1         8             7   
4            5           1         8             4   

                                            Dialogue  
0  I should have known that you would be here...P...  
1  Good evening, Professor Dumbledore. Are the ru...  
2   I'm afraid so, Professor. The good, and the bad.  
3                                       And the boy?  
4                            Hagrid is bringing him.  


ValueError: Value must be a nonnegative integer or None

In [3]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

# Load the dataset
data = pd.read_csv("Dialogue.csv", encoding="latin1")

# Group by 'Dialogue' column and count occurrences
data.groupby('Dialogue').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))

# Remove top and right spines
plt.gca().spines[['top', 'right']].set_visible(False)

# Show the plot
plt.show()


c:\Users\user\anaconda3\Lib\site-packages\IPython\core\pylabtools.py:152: UserWarning: Glyph 128 (\x80) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
c:\Users\user\anaconda3\Lib\site-packages\IPython\core\pylabtools.py:152: UserWarning: Glyph 157 (\x9d) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
c:\Users\user\anaconda3\Lib\site-packages\IPython\core\pylabtools.py:152: UserWarning: Glyph 147 (\x93) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)


KeyboardInterrupt: 

In [21]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Load the dataset using pandas
data = pd.read_csv("Dialogue.csv", encoding="latin1")

# Extract the last column
text_column = data.iloc[:, -1]

# Concatenate all text from the last column into a single string
text = " ".join(text_column.astype(str))

# Initialize the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})  # Add padding token

# Save the text to a temporary file
with open("temp_input.txt", "w") as f:
    f.write(text)

# Create a PyTorch dataset
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="temp_input.txt",  # Provide the path to the temporary file
    block_size=128,
    overwrite_cache=False,  # Set this to True if you want to overwrite the cached dataset
)

# Initialize the GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")

# # Define the training arguments
# training_args = TrainingArguments(
#     output_dir="./fine-tuned-model",
#     overwrite_output_dir=True,
#     num_train_epochs=3,
#     per_device_train_batch_size=4,
#     save_steps=100,
#     save_total_limit=5534,
#     logging_dir="./logs",
#     logging_steps=100,
#     do_train=True
# )

# # Initialize the Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
#     train_dataset=dataset
# )

# # Start training
# trainer.train()

# Function to generate story based on user prompt
def generate_story(user_prompt, max_length=5000, temperature=0.7, top_k=50):
    # Tokenize and encode user prompt
    input_ids = tokenizer.encode(user_prompt, return_tensors="pt")

    # Generate story based on user prompt
    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        temperature=temperature,
        top_k=top_k,
        repetition_penalty=1.2
    )

    # Decode generated story
    generated_story = tokenizer.decode(output[0], skip_special_tokens=True)

    # Add spaces after punctuation marks
    punctuation_marks = [".", ",", "!", "?", ";", ":"]  # Add more if needed
    for mark in punctuation_marks:
        generated_story = generated_story.replace(mark, mark + " ")  # Add space after the punctuation mark

    return generated_story

# Prompt user for input and generate story
user_prompt = input("Enter your story prompt: ")
generated_story = generate_story(user_prompt)

# Display generated story
print("Generated Story:")
print(generated_story)

c:\Users\user\anaconda3\Lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
c:\Users\user\anaconda3\Lib\site-packages\transformers\generation\configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Generated Story:
hello,  PA)
. . .  The first time I saw the new "The Walking Dead" trailer was in a theater at my local mall on Saturday night when it came out and there were people all over town who had seen this movie for years. " - Michael J.  Fox,  AMC's executive producer of original series such as Breaking Bad (2008),  Mad Men (2009).  He said that he has been watching his favorite show since its inception with great interest because they are so different from each other: 

 (via ComicBookMovie )


In [9]:
%pip install spacy

  Obtaining dependency information for spacy from https://files.pythonhosted.org/packages/92/fb/d1f0605e1e8627226c6c96053fe1632e9a04a3fbcd8b5d715528cb95eb97/spacy-3.7.4-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for spacy-legacy<3.1.0,>=3.0.11 from https://files.pythonhosted.org/packages/c3/55/12e842c70ff8828e34e543a2c7176dac4da006ca6901c9e8b43efab8bc6b/spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata
  Obtaining dependency information for spacy-loggers<2.0.0,>=1.0.0 from https://files.pythonhosted.org/packages/33/78/d1a1a026ef3af911159398c939b1509d5c36fe524c7b644f34a5146c4e16/spacy_loggers-1.0.5-py3-none-any.whl.metadata
  Obtaining dependency information for murmurhash<1.1.0,>=0.28.0 from https://files.pythonhosted.org/packages/71/46/af01a20ec368bd9cb49a1d2df15e3eca113bbf6952cc1f2a47f1c6801a7f/murmurhash-1.0.10-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for cymem<2.1.0,>=2.0.2 from https://files.pythonhosted.org/packages/c1/c3/dd